In [17]:
%run TrackEval/scripts/run_kitti_mots.py --USE_PARALLEL False --METRICS HOTA --TRACKERS_TO_EVAL sam


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : C:\Users\Kevin\Documents\Master Related\IS\Thesis\project\TrackEval\error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

KittiMOTS Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : C:\Users\Kevin\Documents\Master Related\IS\Thesis\project\TrackEval\data/gt/kitti/


Count: sam-car                     Dets      GT_Dets   IDs       GT_IDs    
0002                               846       903       124       15        
0006                               552       537       48        11        
0007                               2314      2258      306       53        
0008                               1038      1042      109       21        
0010                               613       602       70        13        
0013                               81        36        36        2         
0014                               392       459       116       14        
0016                               817       834       64        4         
0018                               1304      1358      69        18        
COMBINED                           7957      8029      942       151       

HOTA: sam-pedestrian               HOTA      DetA      AssA      DetRe     DetPr     AssRe     AssPr     LocA      OWTA      HOTA(0)   LocA(0)   HOTALocA(0)
0002 

<Figure size 640x480 with 0 Axes>

In [6]:
%run TrackEval/scripts/run_kitti_mots.py --USE_PARALLEL False --METRICS HOTA --TRACKERS_TO_EVAL track_rcnn


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : C:\Users\Kevin\Documents\Master Related\IS\Thesis\project\TrackEval\error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

KittiMOTS Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : C:\Users\Kevin\Documents\Master Related\IS\Thesis\project\TrackEval\data/gt/kitti/


Count: track_rcnn-car              Dets      GT_Dets   IDs       GT_IDs    
0002                               767       903       20        15        
0006                               528       537       17        11        
0007                               2196      2258      53        53        
0008                               1015      1042      21        21        
0010                               580       602       11        13        
0013                               34        36        6         2         
0014                               401       459       12        14        
0016                               560       834       17        4         
0018                               1329      1358      16        18        
COMBINED                           7410      8029      173       151       

HOTA: track_rcnn-pedestrian        HOTA      DetA      AssA      DetRe     DetPr     AssRe     AssPr     LocA      OWTA      HOTA(0)   LocA(0)   HOTALocA(0)
0002 

<Figure size 640x480 with 0 Axes>

In [14]:
import numpy as np
import os
import cv2
from pycocotools import mask as cocomask
from tqdm import tqdm

def calculate_mask_iou(mask1, mask2):

    mask1_bool = mask1.astype(bool)
    mask2_bool = mask2.astype(bool)

    intersection = np.sum(mask1_bool & mask2_bool)
    union = np.sum(mask1_bool | mask2_bool)

    if union == 0:
        return 0.0
    return intersection / union

def calculate_mask_dice(mask1, mask2):

    mask1_bool = mask1.astype(bool)
    mask2_bool = mask2.astype(bool)

    intersection = np.sum(mask1_bool & mask2_bool)
    # The sum of elements in both masks (2 * intersection + (union - intersection))
    total_elements = np.sum(mask1_bool) + np.sum(mask2_bool)

    if total_elements == 0:
        return 0.0
    return (2.0 * intersection) / total_elements

def parse_kitti_mots_file(filepath):
    """
    Parses a KITTI MOTS .txt file and returns a dictionary of masks per frame.

    Args:
        filepath (str): Path to the KITTI MOTS .txt file.

    Returns:
        dict: A dictionary where keys are frame numbers (int) and values are
              lists of dictionaries. Each inner dictionary contains 'mask',
              'obj_id', 'cls_id', 'height', 'width'.
    """
    frames_data = {}
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split(' ')

            if len(parts) < 6: # Ensure correct data/line structure
                print(f"Incorrect data, skipping line in {filepath}: {line.strip()}")
                continue
            frame_num = int(parts[0])
            obj_id = int(parts[1])
            cls_id = int(parts[2])
            height = int(parts[3])
            width = int(parts[4])
            rle_str = parts[5]

            # Decode RLE string to mask
            rle = {'size': [height, width], 'counts': rle_str.encode('utf-8')}
            
            # Error handling for bad rle string
            try:
                mask = cocomask.decode(rle)
            except Exception as e:
                print(f"Could not decode RLE in frame {frame_num} of {filepath}: {e}. Skipping mask.")
                continue # Skip

            if frame_num not in frames_data:
                frames_data[frame_num] = []
            frames_data[frame_num].append({
                'mask': mask.astype(bool),
                'obj_id': obj_id,
                'cls_id': cls_id,
                'height': height,
                'width': width
            })
    return frames_data

def evaluate_segmentation_quality(predictions_root, ground_truth_root, target_class_id=1):
    #Evaluates segmentation quality (IoU and Dice) for all 'car' instances.

    all_iou_scores = []
    all_dice_scores = []

    prediction_files = sorted([f for f in os.listdir(predictions_root) if f.endswith('.txt')])

    # Iterate through prediction files and check for corresponding ground truth files
    for pred_filename in tqdm(prediction_files, desc="Evaluating files"):
        pred_file_path = os.path.join(predictions_root, pred_filename)
        gt_file_path = os.path.join(ground_truth_root, pred_filename)

        # Only evaluate if both prediction and ground truth files exist
        if not os.path.exists(pred_file_path):
            print(f"Warning: Prediction file not found: {pred_file_path}. Skipping.")
            continue
        if not os.path.exists(gt_file_path):
            print(f"Warning: Corresponding ground truth file not found for {pred_filename}: {gt_file_path}. Skipping.")
            continue

        predicted_frames = parse_kitti_mots_file(pred_file_path)
        gt_frames = parse_kitti_mots_file(gt_file_path)

        # Iterate through frames present in ground truth
        for frame_num in sorted(gt_frames.keys()):
            gt_objects = [obj for obj in gt_frames[frame_num] if obj['cls_id'] == target_class_id]
            predicted_objects = predicted_frames.get(frame_num, [])

            matched_gt_indices = set()

            for pred_obj in predicted_objects:
                # Only evaluate if the predicted object is of the target class (e.g., 'car')
                if pred_obj['cls_id'] != target_class_id:
                    continue

                best_iou = 0.0
                best_gt_idx = -1

                for gt_idx, gt_obj in enumerate(gt_objects):
                    if gt_idx in matched_gt_indices:
                        continue # Skip already matched GT objects

                    # Calculate IoU for matching
                    iou = calculate_mask_iou(pred_obj['mask'], gt_obj['mask'])

                    if iou > best_iou: # Find the best matching ground truth for this prediction
                        best_iou = iou
                        best_gt_idx = gt_idx

                # If a good enough match is found, record the scores
                if best_iou > 0.5: # Tune this threshold for matching
                    all_iou_scores.append(best_iou)
                    all_dice_scores.append(calculate_mask_dice(pred_obj['mask'], gt_objects[best_gt_idx]['mask']))
                    matched_gt_indices.add(best_gt_idx)

    return all_iou_scores, all_dice_scores


In [16]:
if __name__ == "__main__":
    
    predictions_base_dir = "TrackEval/data/trackers/kitti/kitti_mots_val/sam/data"
    ground_truth_base_dir = "TrackEval/data/gt/kitti/kitti_mots_val/label_02"

    car_class_id = 1

    print(f"Evaluation for class ID: {car_class_id} ('Car')...")
    iou_scores, dice_scores = evaluate_segmentation_quality(
        predictions_base_dir,
        ground_truth_base_dir,
        target_class_id=car_class_id
    )

    if iou_scores:
        mean_iou = np.mean(iou_scores)
        mean_dice = np.mean(dice_scores)
        print(f"Evaluation Results for Class ID {car_class_id} ('Car'):")
        print(f"Total matched instances: {len(iou_scores)}")
        print(f"Mean IoU: {mean_iou:.4f}")
        print(f"Mean Dice Coefficient(F-1 Score): {mean_dice:.4f}")
    else:
        print("Failed")

Starting segmentation evaluation for class ID: 1 ('Car')...


Evaluating files: 100%|██████████| 9/9 [00:28<00:00,  3.17s/it]


Evaluation Results for Class ID 1 ('Car'):
  Total matched instances: 7332
  Mean IoU: 0.8857
  Mean Dice Coefficient: 0.9368
